# 陳時中、黃珊珊、蔣萬安市長候選人大PK

Penalty Kick or Player Kill for Taipei city mayor candidates--Chen, Huang, and Jiang

# Load Data 

In [1]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('./now_news_preprocessed.csv',sep='|')

In [2]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,column/_20220505_1001,2022-05-05,焦點,配發率32%！富邦金每股配4元股利、現金股利歷史新高,富邦金控今（5）日公告，董事會決議通過擬配發普通股每股現金股利3.5元及股票股利0.5元，合...,0.0,"['富邦金這次同時配發股票股利每股0.5元', '包含富邦人壽上繳155.78億元、台北富邦...","[('股利', 18), ('富邦', 12), ('現金', 9), ('股票', 5),...","['富邦', '金控', '今', '（5', '）', '日', '公告', '，', '...","['富邦', '金控', '董事會', '通過', '普通股', '現金', '股利', '...","[NerToken(word='富邦金控', ner='ORG', idx=(0, 4)),...","[('富邦', 'Nb'), ('金控', 'Na'), ('今', 'Nd'), ('（5...",https://www.nownews.com/news/5796259,https://media.nownews.com/nn_media/thumbnail/2...


In [3]:
df.shape

(3671, 14)

# Step 1: Filter news articles using the following function

In [4]:
from datetime import datetime, timedelta

In [5]:
from datetime import datetime, timedelta
# Searching keywords from "content" column or tokens_v2
def filter_df_via_tokens(df, query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.tokens_v2.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.tokens_v2.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


In [6]:
from datetime import datetime, timedelta
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_df_via_content(df, query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


# Step 2: Sentimental polarity score計算整體情緒分數(影響力)

In [7]:
# sentimental polarity score
def get_article_sentiment(df_query):
    # df_query = df[df['tokens'].str.contains(query_key)]
    sentiCount = {'pos': 0, 'neg': 0, 'obj': 0}
    sentiPercnt = {'pos': 0, 'neg': 0, 'obj': 0}
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # 判斷文章的情緒極性
        if senti >= 0.75:
            sentiCount['pos'] += 1
        elif senti <= 0.4:
            sentiCount['neg'] += 1
        else:
            sentiCount['obj'] += 1
    for polar in sentiCount :
        sentiPercnt[polar]=int(sentiCount[polar]/numberOfArticle*100)
        #sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle,2)
    return sentiCount, sentiPercnt


In [8]:
query_keywords = ['黃珊珊']
weeks=12
cond='or'
weeks=12
cate='全部'
df_query = filter_df_via_tokens(df, query_keywords, cond, cate, weeks)
sentiCount, sentiPercnt = get_article_sentiment(df_query)

In [9]:
sentiPercnt

{'pos': 2, 'neg': 97, 'obj': 0}

In [10]:
    # sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
    #                                            str(sentiPercnt['neg']))
[sentiPercnt[p] for p in ['pos', 'obj', 'neg']]

[2, 0, 97]

# Step 2: Count categorical frequency and occurrence(number of articles)

    Count how many articles containing these keywords計算各類別多少篇文章提到該關鍵字
    Count how many times these keywords were mentioned in each category計算各類別出現關鍵字次數

In [11]:
# **計算各類別多少篇文章提到該關鍵字
# **計算各類別出現關鍵字次數

news_categories = ['焦點', '要聞', '即時', '娛樂', '新奇', '生活', '財經', '專題', '全球', '運動', '全部']


def count_keyword(df_query, query_keywords):

    cate_occurrence = {}
    cate_freq = {}
    
    for cate in news_categories:
        cate_occurrence[cate] = 0
        cate_freq[cate] = 0

    for idx, row in df_query.iterrows():
        # count the number of articles
        cate_occurrence[row.category] += 1
        cate_occurrence['全部'] += 1
        # count the keyword frequency
        tokens = eval(row.tokens_v2)
        # 計算這一篇文章的tokens中重複含有多少個這些關鍵字(頻率)
        freq = len([word for word in tokens if (word in query_keywords)]) 
        cate_freq[row.category] += freq # 在該新聞類別中累計頻率
        cate_freq['全部'] += freq  # 在"全部"類別中累計頻率

    total_articles = cate_occurrence['全部']  # len(df_query)
    total_frequency = cate_freq['全部']


    
    return cate_freq, cate_occurrence, total_articles, total_frequency

In [12]:
query_keywords = ['黃珊珊']
weeks=12
cond='or'
weeks=12
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'焦點': 37,
  '要聞': 82,
  '即時': 17,
  '娛樂': 0,
  '新奇': 0,
  '生活': 52,
  '財經': 1,
  '專題': 0,
  '全球': 0,
  '運動': 0,
  '全部': 189},
 {'焦點': 8,
  '要聞': 21,
  '即時': 7,
  '娛樂': 0,
  '新奇': 0,
  '生活': 12,
  '財經': 1,
  '專題': 0,
  '全球': 0,
  '運動': 0,
  '全部': 49},
 49,
 189)

# Step 3: Caclulate daily-basis number of articles which contain the query keywords

計算以時間為基礎的被報導的篇數

In [13]:
def get_keyword_occurrence_time_series(df_query):


    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)
    

    # 設定時間欄位'date_index', 次數freq欄位，將每一篇的時間寫到'date_index'欄位，將freq設定為1 ==> 進行groupby之後就可以合併日期與加總freq次數
    query_freq = pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'freq':[1 for _ in range(len(df_query))]})

    # 開始時間、結束時間兩項必須也加入到query_freq，計算次數時才會有完整的時間軸，否則時間軸長度會因為新聞時間不同，導致時間軸不一致
    dt_start_date = datetime.strptime(start_date, '%Y-%m-%d')
    dt_end_date = datetime.strptime(end_date, '%Y-%m-%d')   

    query_freq = query_freq.append({'date_index': dt_start_date, 'freq': 0}, ignore_index=True)
    query_freq = query_freq.append({'date_index': dt_end_date, 'freq': 0}, ignore_index=True)

    # 透過groupby就可以合併日期與加總freq次數
    freq_data_group = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()

    # 'date_index'為index(索引)，將其變成欄位名稱，inplace=True表示原始檔案會被異動
    freq_data_group.reset_index(inplace=True)
    # freq_data_group = freq_data_group.reset_index() # 這樣也可以得到同樣結果

    # 只有頻率次數值y, 沒有時間變數x, 計算相關係數時會用到
    # freq_data = freq_data_group.freq.to_list()

    # 有時間變數x,y，用於畫趨勢線圖
    line_xy_data = [{'x':date.strftime('%Y-%m-%d'),'y':freq} for date, freq in zip(freq_data_group.date_index,freq_data_group.freq)]

    return line_xy_data


In [14]:
# Alternative approach for reference
# def get_key_time_freq(df_query):
#     date_samples = df_query.date
#     query_freq = pd.DataFrame({'date_index': pd.to_datetime(date_samples), 'freq': [1 for _ in range(len(df_query))]})
#     data = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()
#     time_data = []
#     for i, idx in enumerate(data.index):
#         row = {'x': idx.strftime('%Y-%m-%d'), 'y': int(data.iloc[i].freq)}
#         time_data.append(row)
#     return time_data
# get_key_time_freq(df_query)

In [15]:
query_keywords = ['黃珊珊']
weeks=12
cond='or'
weeks=12
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
get_keyword_occurrence_time_series(df_query)


C:\Users\Calvin\AppData\Local\Temp\ipykernel_45104\908549974.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  query_freq = query_freq.append({'date_index': dt_start_date, 'freq': 0}, ignore_index=True)
C:\Users\Calvin\AppData\Local\Temp\ipykernel_45104\908549974.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  query_freq = query_freq.append({'date_index': dt_end_date, 'freq': 0}, ignore_index=True)


[{'x': '2022-04-08', 'y': 0},
 {'x': '2022-04-09', 'y': 0},
 {'x': '2022-04-10', 'y': 0},
 {'x': '2022-04-11', 'y': 0},
 {'x': '2022-04-12', 'y': 0},
 {'x': '2022-04-13', 'y': 0},
 {'x': '2022-04-14', 'y': 0},
 {'x': '2022-04-15', 'y': 0},
 {'x': '2022-04-16', 'y': 0},
 {'x': '2022-04-17', 'y': 0},
 {'x': '2022-04-18', 'y': 0},
 {'x': '2022-04-19', 'y': 0},
 {'x': '2022-04-20', 'y': 0},
 {'x': '2022-04-21', 'y': 0},
 {'x': '2022-04-22', 'y': 0},
 {'x': '2022-04-23', 'y': 0},
 {'x': '2022-04-24', 'y': 0},
 {'x': '2022-04-25', 'y': 0},
 {'x': '2022-04-26', 'y': 0},
 {'x': '2022-04-27', 'y': 0},
 {'x': '2022-04-28', 'y': 0},
 {'x': '2022-04-29', 'y': 0},
 {'x': '2022-04-30', 'y': 0},
 {'x': '2022-05-01', 'y': 0},
 {'x': '2022-05-02', 'y': 0},
 {'x': '2022-05-03', 'y': 0},
 {'x': '2022-05-04', 'y': 0},
 {'x': '2022-05-05', 'y': 0},
 {'x': '2022-05-06', 'y': 1},
 {'x': '2022-05-07', 'y': 0},
 {'x': '2022-05-08', 'y': 0},
 {'x': '2022-05-09', 'y': 10},
 {'x': '2022-05-10', 'y': 6},
 {'x': '2

In [16]:
query_keywords = ['陳時中']
weeks=12
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
get_keyword_occurrence_time_series(df_query)


C:\Users\Calvin\AppData\Local\Temp\ipykernel_45104\908549974.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  query_freq = query_freq.append({'date_index': dt_start_date, 'freq': 0}, ignore_index=True)
C:\Users\Calvin\AppData\Local\Temp\ipykernel_45104\908549974.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  query_freq = query_freq.append({'date_index': dt_end_date, 'freq': 0}, ignore_index=True)


[{'x': '2022-04-08', 'y': 0},
 {'x': '2022-04-09', 'y': 0},
 {'x': '2022-04-10', 'y': 0},
 {'x': '2022-04-11', 'y': 0},
 {'x': '2022-04-12', 'y': 0},
 {'x': '2022-04-13', 'y': 0},
 {'x': '2022-04-14', 'y': 0},
 {'x': '2022-04-15', 'y': 0},
 {'x': '2022-04-16', 'y': 0},
 {'x': '2022-04-17', 'y': 0},
 {'x': '2022-04-18', 'y': 0},
 {'x': '2022-04-19', 'y': 0},
 {'x': '2022-04-20', 'y': 0},
 {'x': '2022-04-21', 'y': 0},
 {'x': '2022-04-22', 'y': 0},
 {'x': '2022-04-23', 'y': 0},
 {'x': '2022-04-24', 'y': 0},
 {'x': '2022-04-25', 'y': 0},
 {'x': '2022-04-26', 'y': 0},
 {'x': '2022-04-27', 'y': 0},
 {'x': '2022-04-28', 'y': 0},
 {'x': '2022-04-29', 'y': 0},
 {'x': '2022-04-30', 'y': 0},
 {'x': '2022-05-01', 'y': 0},
 {'x': '2022-05-02', 'y': 0},
 {'x': '2022-05-03', 'y': 0},
 {'x': '2022-05-04', 'y': 0},
 {'x': '2022-05-05', 'y': 5},
 {'x': '2022-05-06', 'y': 7},
 {'x': '2022-05-07', 'y': 0},
 {'x': '2022-05-08', 'y': 0},
 {'x': '2022-05-09', 'y': 58},
 {'x': '2022-05-10', 'y': 10},
 {'x': '

# Step 4: Process PK data

In [19]:
# 定義PK對象
# 通常會多個關鍵字代表該PK對象，['柯文哲','柯p','柯P']
list_pkNames = ['陳時中', '黃珊珊', '蔣萬安']
list_pkKeywordSet = [['陳時中'], ['黃珊珊'], ['蔣萬安']]


# 線條顏色
list_colors = ["rgba(0,128,0,0.2)",'rgba(0,255,255,0.2)','rgba(0,0,255,0.2)']
#list_colors = ['green', 'red', 'blue']

# 人頭圖案
list_photos = [
    "https://upload.wikimedia.org/wikipedia/commons/c/c0/%E8%A1%9B%E7%94%9F%E7%A6%8F%E5%88%A9%E9%83%A8%E9%83%A8%E9%95%B7%E9%99%B3%E6%99%82%E4%B8%AD.jpg",
    "https://councilorwatch.tw/images/councillors/female/230.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/9/93/%E8%94%A3%E8%90%AC%E5%AE%89%E5%A7%94%E5%93%A1.jpg"
]

# 準備長條圖 線圖 總篇數 總次數等數據
list_freq_daily_line_chart = []
list_freq_news_category = []
list_sentimentInfo = []

list_total_articles=[]
list_total_frequency=[]

weeks = 12
cate = "全部"
cond = 'or'

selectedCategories = ['全部', '焦點', '要聞', '即時', '生活']
#news_categories = ['焦點', '要聞', '即時', '娛樂', '新奇', '生活', '財經', '專題', '全球', '運動']
# selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']

for query_keywords in list_pkKeywordSet:

    # Filter news   
    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
    # Get line chart data
    line_xy_data = get_keyword_occurrence_time_series(df_query)
    list_freq_daily_line_chart.append(line_xy_data)
    # Get bar chart data, total articals and frequecy 
    cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(df_query, query_keywords)
    list_total_articles.append(total_articles)
    list_total_frequency.append(total_frequency)


    # We select these categories to display: ['全部', '政治', '兩岸', '產經', '生活', '社會']
    cate_freq = [cate_occurrence[k] for k in selectedCategories]
    list_freq_news_category.append(cate_freq)

    # Get sentiment information
    sentiCount, sentiPercnt = get_article_sentiment(df_query)
    #sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
    #                                            str(sentiPercnt['neg']))
    senti = [sentiPercnt[p] for p in ['pos','obj','neg']] # 只要取用pos,obj, neg的情緒數字篇數百分比
    list_sentimentInfo.append(senti)

# We need all the following data to display on our frontend page 有一大堆數據要送到前端去展示啊
pk_data =  {'list_freq_daily_line_chart': list_freq_daily_line_chart,
           'list_pkNames': list_pkNames,
           'list_colors': list_colors,
           'list_photos': list_photos,
           'list_freq_news_category': list_freq_news_category,
           'list_category': selectedCategories,
           'list_sentiInfo': list_sentimentInfo,
           'list_total_articles': list_total_articles,
           'list_total_frequency': list_total_frequency
           }

C:\Users\Calvin\AppData\Local\Temp\ipykernel_45104\908549974.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  query_freq = query_freq.append({'date_index': dt_start_date, 'freq': 0}, ignore_index=True)
C:\Users\Calvin\AppData\Local\Temp\ipykernel_45104\908549974.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  query_freq = query_freq.append({'date_index': dt_end_date, 'freq': 0}, ignore_index=True)
C:\Users\Calvin\AppData\Local\Temp\ipykernel_45104\908549974.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  query_freq = query_freq.append({'date_index': dt_start_date, 'freq': 0}, ignore_index=True)
C:\Users\Calvin\AppData\Local\Temp\ipykernel_45104\908549974.py:22: FutureWarning: The frame.append method is depreca

In [20]:
pk_data

{'list_freq_daily_line_chart': [[{'x': '2022-04-08', 'y': 0},
   {'x': '2022-04-09', 'y': 0},
   {'x': '2022-04-10', 'y': 0},
   {'x': '2022-04-11', 'y': 0},
   {'x': '2022-04-12', 'y': 0},
   {'x': '2022-04-13', 'y': 0},
   {'x': '2022-04-14', 'y': 0},
   {'x': '2022-04-15', 'y': 0},
   {'x': '2022-04-16', 'y': 0},
   {'x': '2022-04-17', 'y': 0},
   {'x': '2022-04-18', 'y': 0},
   {'x': '2022-04-19', 'y': 0},
   {'x': '2022-04-20', 'y': 0},
   {'x': '2022-04-21', 'y': 0},
   {'x': '2022-04-22', 'y': 0},
   {'x': '2022-04-23', 'y': 0},
   {'x': '2022-04-24', 'y': 0},
   {'x': '2022-04-25', 'y': 0},
   {'x': '2022-04-26', 'y': 0},
   {'x': '2022-04-27', 'y': 0},
   {'x': '2022-04-28', 'y': 0},
   {'x': '2022-04-29', 'y': 0},
   {'x': '2022-04-30', 'y': 0},
   {'x': '2022-05-01', 'y': 0},
   {'x': '2022-05-02', 'y': 0},
   {'x': '2022-05-03', 'y': 0},
   {'x': '2022-05-04', 'y': 0},
   {'x': '2022-05-05', 'y': 5},
   {'x': '2022-05-06', 'y': 7},
   {'x': '2022-05-07', 'y': 0},
   {'x': '

# Step 5: Save PK data to csv file

In [21]:
df_data_pk = pd.DataFrame(list(pk_data.items()),columns=['name','value'])

In [22]:
df_data_pk

,name,value
0,list_freq_daily_line_chart,"[[{'x': '2022-04-08', 'y': 0}, {'x': '2022-04-..."
1,list_pkNames,"[陳時中, 黃珊珊, 蔣萬安]"
2,list_colors,"[rgba(0,128,0,0.2), rgba(0,255,255,0.2), rgba(..."
3,list_photos,[https://upload.wikimedia.org/wikipedia/common...
4,list_freq_news_category,"[[212, 41, 74, 44, 42], [49, 8, 21, 7, 12], [2..."
5,list_category,"[全部, 焦點, 要聞, 即時, 生活]"
6,list_sentiInfo,"[[4, 0, 94], [2, 0, 97], [16, 0, 84]]"
7,list_total_articles,"[212, 49, 25]"
8,list_total_frequency,"[551, 189, 123]"


In [23]:
## 存成csv格式檔案
df_data_pk.to_csv('pk_taipei_mayor_election.csv',sep=',', index=None)

### Alternative way: using zip

In [24]:
k=list(pk_data.keys())
v=list(pk_data.values())

In [25]:
#list(zip(k,v))

In [26]:
df_data_pk = pd.DataFrame(list(zip(k,v)),columns=['name','value'])
df_data_pk

,name,value
0,list_freq_daily_line_chart,"[[{'x': '2022-04-08', 'y': 0}, {'x': '2022-04-..."
1,list_pkNames,"[陳時中, 黃珊珊, 蔣萬安]"
2,list_colors,"[rgba(0,128,0,0.2), rgba(0,255,255,0.2), rgba(..."
3,list_photos,[https://upload.wikimedia.org/wikipedia/common...
4,list_freq_news_category,"[[212, 41, 74, 44, 42], [49, 8, 21, 7, 12], [2..."
5,list_category,"[全部, 焦點, 要聞, 即時, 生活]"
6,list_sentiInfo,"[[4, 0, 94], [2, 0, 97], [16, 0, 84]]"
7,list_total_articles,"[212, 49, 25]"
8,list_total_frequency,"[551, 189, 123]"


## Read csv file and convert to dict format

In [27]:
df_data_pk = pd.read_csv('pk_taipei_mayor_election.csv')

In [28]:
df_data_pk 

,name,value
0,list_freq_daily_line_chart,"[[{'x': '2022-04-08', 'y': 0}, {'x': '2022-04-..."
1,list_pkNames,"['陳時中', '黃珊珊', '蔣萬安']"
2,list_colors,"['rgba(0,128,0,0.2)', 'rgba(0,255,255,0.2)', '..."
3,list_photos,['https://upload.wikimedia.org/wikipedia/commo...
4,list_freq_news_category,"[[212, 41, 74, 44, 42], [49, 8, 21, 7, 12], [2..."
5,list_category,"['全部', '焦點', '要聞', '即時', '生活']"
6,list_sentiInfo,"[[4, 0, 94], [2, 0, 97], [16, 0, 84]]"
7,list_total_articles,"[212, 49, 25]"
8,list_total_frequency,"[551, 189, 123]"


In [29]:
df_data_pk.values


array([['list_freq_daily_line_chart',
        "[[{'x': '2022-04-08', 'y': 0}, {'x': '2022-04-09', 'y': 0}, {'x': '2022-04-10', 'y': 0}, {'x': '2022-04-11', 'y': 0}, {'x': '2022-04-12', 'y': 0}, {'x': '2022-04-13', 'y': 0}, {'x': '2022-04-14', 'y': 0}, {'x': '2022-04-15', 'y': 0}, {'x': '2022-04-16', 'y': 0}, {'x': '2022-04-17', 'y': 0}, {'x': '2022-04-18', 'y': 0}, {'x': '2022-04-19', 'y': 0}, {'x': '2022-04-20', 'y': 0}, {'x': '2022-04-21', 'y': 0}, {'x': '2022-04-22', 'y': 0}, {'x': '2022-04-23', 'y': 0}, {'x': '2022-04-24', 'y': 0}, {'x': '2022-04-25', 'y': 0}, {'x': '2022-04-26', 'y': 0}, {'x': '2022-04-27', 'y': 0}, {'x': '2022-04-28', 'y': 0}, {'x': '2022-04-29', 'y': 0}, {'x': '2022-04-30', 'y': 0}, {'x': '2022-05-01', 'y': 0}, {'x': '2022-05-02', 'y': 0}, {'x': '2022-05-03', 'y': 0}, {'x': '2022-05-04', 'y': 0}, {'x': '2022-05-05', 'y': 5}, {'x': '2022-05-06', 'y': 7}, {'x': '2022-05-07', 'y': 0}, {'x': '2022-05-08', 'y': 0}, {'x': '2022-05-09', 'y': 58}, {'x': '2022-05-10', 'y

In [30]:
# Convert to dictionary format
# data = dict(list(df_data_pk.values))

## Convert to dictionary format

In [31]:
data={}
for k, v in df_data_pk.values:
    #print(eval(v))
    data[k]=eval(v)

In [32]:
# Another approach to get the sampe result
data={}
for k,v in zip(df_data_pk.name, df_data_pk.value):
    data[k]=eval(v)
    

In [33]:
data

{'list_freq_daily_line_chart': [[{'x': '2022-04-08', 'y': 0},
   {'x': '2022-04-09', 'y': 0},
   {'x': '2022-04-10', 'y': 0},
   {'x': '2022-04-11', 'y': 0},
   {'x': '2022-04-12', 'y': 0},
   {'x': '2022-04-13', 'y': 0},
   {'x': '2022-04-14', 'y': 0},
   {'x': '2022-04-15', 'y': 0},
   {'x': '2022-04-16', 'y': 0},
   {'x': '2022-04-17', 'y': 0},
   {'x': '2022-04-18', 'y': 0},
   {'x': '2022-04-19', 'y': 0},
   {'x': '2022-04-20', 'y': 0},
   {'x': '2022-04-21', 'y': 0},
   {'x': '2022-04-22', 'y': 0},
   {'x': '2022-04-23', 'y': 0},
   {'x': '2022-04-24', 'y': 0},
   {'x': '2022-04-25', 'y': 0},
   {'x': '2022-04-26', 'y': 0},
   {'x': '2022-04-27', 'y': 0},
   {'x': '2022-04-28', 'y': 0},
   {'x': '2022-04-29', 'y': 0},
   {'x': '2022-04-30', 'y': 0},
   {'x': '2022-05-01', 'y': 0},
   {'x': '2022-05-02', 'y': 0},
   {'x': '2022-05-03', 'y': 0},
   {'x': '2022-05-04', 'y': 0},
   {'x': '2022-05-05', 'y': 5},
   {'x': '2022-05-06', 'y': 7},
   {'x': '2022-05-07', 'y': 0},
   {'x': '

: 

# views.py in Django

In [32]:
from django.http import JsonResponse
import pandas as pd
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt

def load_data_pk():
    # Read data from csv file
    df_data_pk = pd.read_csv('app_taipei_mayor/dataset/pk_taipei_mayor_election.csv')
    
    global data
    data={}
    for k,v in zip(df_data_pk.name, df_data_pk.value):
        data[k]=eval(v)
    
    # 沒用到的變數刪除之
    del df_data_pk

# load pk data
load_data_pk()

def home(request):
    return render(request,'app_taipei_mayor/home.html')

# csrf_exempt is used for POST
# 單獨指定這一支程式忽略csrf驗證
@csrf_exempt
def api_get_taipei_mayor_data(request):
    return JsonResponse(data)

print('app taipei mayor leaderboard was loaded!')


FileNotFoundError: [Errno 2] No such file or directory: 'app_taipei_mayor/dataset/pk_taipei_mayor_election.csv'

#  For reference: alternative aprroach to process response data with json format


In [ ]:
# 個別包含完整的資料
list_pkNames = ['陳時中', '黃珊珊', '蔣萬安']
list_pkKeywordSet = [['陳時中'], ['黃珊珊'], ['蔣萬安']]

# pkName = ['柯文哲', '蔡英文', '韓國瑜']
# pkset = [['柯文哲','柯p','柯P'], ['蔡英文'], ['韓國瑜']]

#list_colors = ['green', 'red', 'blue']
list_colors = ["rgba(0,128,0,0.2)",'rgba(0,255,255,0.2)','rgba(0,0,255,0.2)']

list_photos = [
    "https://upload.wikimedia.org/wikipedia/commons/c/c0/%E8%A1%9B%E7%94%9F%E7%A6%8F%E5%88%A9%E9%83%A8%E9%83%A8%E9%95%B7%E9%99%B3%E6%99%82%E4%B8%AD.jpg",
    "https://councilorwatch.tw/images/councillors/female/230.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/9/93/%E8%94%A3%E8%90%AC%E5%AE%89%E5%A7%94%E5%93%A1.jpg"
]


def get_pk_data(name, query_keywords, photo, color):

    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)

    line_xy_data = get_keyword_occurrence_time_series(df_query)

    cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(
        df_query, query_keywords)
    selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
    # selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']
    # We select these categories to display: ['全部', '政治', '兩岸', '產經', '生活', '社會']
    freq_selected_categories = [cate_occurrence[k] for k in selectedCategories]

    # sentiment information
    sentiCount, sentiPercnt = get_article_sentiment(df_query)
    # sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
    #                                            str(sentiPercnt['neg']))
    sentiment = [sentiPercnt[p] for p in ['pos', 'obj', 'neg']]

    pk_data = {'line_xy_data': line_xy_data,
               'name': name,
               'color': color,
               'photo': photo,
               'freq_news_category': freq_selected_categories,
               'category': selectedCategories,
               'sentiment': sentiment,
               'total_articles': total_articles,
               'total_frequency': total_frequency,
               }
    return pk_data

pk_data_dict = {}
for i in range(len(list_pkNames)):
    name = list_pkNames[i]
    query_keywords = list_pkKeywordSet[i]
    photo = list_photos[i]
    color = list_colors[i]
    pk_data_dict[name] = get_pk_data(name, query_keywords, photo, color)



In [ ]:
pk_data_dict

{'陳時中': {'line_xy_data': [{'x': '2022-01-29', 'y': 5},
   {'x': '2022-01-30', 'y': 5},
   {'x': '2022-01-31', 'y': 2},
   {'x': '2022-02-01', 'y': 0},
   {'x': '2022-02-02', 'y': 0},
   {'x': '2022-02-03', 'y': 0},
   {'x': '2022-02-04', 'y': 6},
   {'x': '2022-02-05', 'y': 13},
   {'x': '2022-02-06', 'y': 3},
   {'x': '2022-02-07', 'y': 11},
   {'x': '2022-02-08', 'y': 12},
   {'x': '2022-02-09', 'y': 13},
   {'x': '2022-02-10', 'y': 8},
   {'x': '2022-02-11', 'y': 10},
   {'x': '2022-02-12', 'y': 10},
   {'x': '2022-02-13', 'y': 3},
   {'x': '2022-02-14', 'y': 10},
   {'x': '2022-02-15', 'y': 15},
   {'x': '2022-02-16', 'y': 13},
   {'x': '2022-02-17', 'y': 13},
   {'x': '2022-02-18', 'y': 14},
   {'x': '2022-02-19', 'y': 10},
   {'x': '2022-02-20', 'y': 2},
   {'x': '2022-02-21', 'y': 11},
   {'x': '2022-02-22', 'y': 11},
   {'x': '2022-02-23', 'y': 10},
   {'x': '2022-02-24', 'y': 12},
   {'x': '2022-02-25', 'y': 6},
   {'x': '2022-02-26', 'y': 4},
   {'x': '2022-02-27', 'y': 1},
 

### Save file using json format

In [ ]:

import json
# Serialize data into file:
json.dump( pk_data_dict, open( "pk_taipei_mayor_election.json", 'w' ) )

# Read data from file:
data = json.load( open( "pK_taipei_mayor_election.json" ) )
data

{'陳時中': {'line_xy_data': [{'x': '2022-01-29', 'y': 5},
   {'x': '2022-01-30', 'y': 5},
   {'x': '2022-01-31', 'y': 2},
   {'x': '2022-02-01', 'y': 0},
   {'x': '2022-02-02', 'y': 0},
   {'x': '2022-02-03', 'y': 0},
   {'x': '2022-02-04', 'y': 6},
   {'x': '2022-02-05', 'y': 13},
   {'x': '2022-02-06', 'y': 3},
   {'x': '2022-02-07', 'y': 11},
   {'x': '2022-02-08', 'y': 12},
   {'x': '2022-02-09', 'y': 13},
   {'x': '2022-02-10', 'y': 8},
   {'x': '2022-02-11', 'y': 10},
   {'x': '2022-02-12', 'y': 10},
   {'x': '2022-02-13', 'y': 3},
   {'x': '2022-02-14', 'y': 10},
   {'x': '2022-02-15', 'y': 15},
   {'x': '2022-02-16', 'y': 13},
   {'x': '2022-02-17', 'y': 13},
   {'x': '2022-02-18', 'y': 14},
   {'x': '2022-02-19', 'y': 10},
   {'x': '2022-02-20', 'y': 2},
   {'x': '2022-02-21', 'y': 11},
   {'x': '2022-02-22', 'y': 11},
   {'x': '2022-02-23', 'y': 10},
   {'x': '2022-02-24', 'y': 12},
   {'x': '2022-02-25', 'y': 6},
   {'x': '2022-02-26', 'y': 4},
   {'x': '2022-02-27', 'y': 1},
 